<a href="https://colab.research.google.com/github/HaTT2018/Air_Traffic/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Air Traffic Data

In [5]:
!pip install -U --no-cache-dir gdown --pre

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=26809dfba5f382282566fba7b878abe4815758515837ea988404abd6358e4896
  Stored in directory: /tmp/pip-ephem-wheel-cache-439hpsc3/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [6]:
!gdown --id 12UB_JFNcH2-FTNVgcCcGQD5CD0zYcLUP --no-cookies

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=12UB_JFNcH2-FTNVgcCcGQD5CD0zYcLUP
To: /content/718598182_T_DB1B_MARKET.csv
100% 455M/455M [00:03<00:00, 152MB/s]


# Load Data